In [1]:
# @title 🛠️ Step 1: Find Your Valid Model & Reset Libraries
# Uninstall broken versions and install fresh
!pip uninstall -y google-generativeai langchain-google-genai langgraph langchain
!pip install -U google-generativeai langchain-google-genai langgraph langchain_community duckduckgo-search requests

import requests
import os
import getpass

# 1. Get API Key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google Gemini API Key: ")

api_key = os.environ["GOOGLE_API_KEY"]

# 2. RAW API Query (Bypassing Python SDK to get the TRUTH)
print("\n📡 Connecting to Google API servers directly...")
url = f"https://generativelanguage.googleapis.com/v1beta/models?key={api_key}"
response = requests.get(url)

valid_model = None

if response.status_code == 200:
    data = response.json()
    print("\n✅ AVAILABLE MODELS (Supported by your Key):")

    # Filter for models that support 'generateContent'
    chat_models = [
        m for m in data.get('models', [])
        if 'generateContent' in m.get('supportedGenerationMethods', [])
    ]

    # Prioritize Stable Flash -> Stable Pro -> Legacy
    # We look for '002' or '001' specifically to avoid 404s on aliases
    priority_order = [
        "gemini-1.5-flash-002",
        "gemini-1.5-flash-001",
        "gemini-1.5-pro-002",
        "gemini-1.5-pro-001",
        "gemini-1.0-pro"
    ]

    # Create a map for easy lookup
    available_names = [m['name'].replace("models/", "") for m in chat_models]

    for p in priority_order:
        if p in available_names:
            valid_model = p
            break

    # Fallback: Just take the first Flash model found
    if not valid_model:
        for name in available_names:
            if "flash" in name and "exp" not in name: # Avoid experimental (quota issues)
                valid_model = name
                break

    # Final Fallback: Take ANYTHING
    if not valid_model and available_names:
        valid_model = available_names[0]

    print(f"🎯 AUTO-SELECTED BEST MODEL: {valid_model}")

else:
    print(f"❌ API Error: {response.status_code} - {response.text}")

# Store for next cell
if valid_model:
    os.environ["VALID_MODEL_ID"] = valid_model
    print("\n✅ Setup Complete. Run the next cell.")
else:
    print("\n⚠️ CRITICAL: No models found. Check your API Key permissions.")

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: langchain-google-genai 2.0.10
Uninstalling langchain-google-genai-2.0.10:
  Successfully uninstalled langchain-google-genai-2.0.10
Found existing installation: langgraph 1.0.1
Uninstalling langgraph-1.0.1:
  Successfully uninstalled langgraph-1.0.1
Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached langchain_google_genai-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langgraph-1.0.3-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a whil

Enter your Google Gemini API Key: ··········

📡 Connecting to Google API servers directly...

✅ AVAILABLE MODELS (Supported by your Key):
🎯 AUTO-SELECTED BEST MODEL: gemini-2.5-flash

✅ Setup Complete. Run the next cell.


In [2]:
# @title 🚀 Step 2: Run "DealCloser" Agent
# This uses the VALID_MODEL_ID found above to prevent 404s.

import os
import time
from typing import TypedDict, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# --- 1. CONFIGURATION ---
# Retrieve the validated model ID
active_model = os.environ.get("VALID_MODEL_ID", "gemini-1.5-flash")
print(f"🤖 Initializing Agent with Model: {active_model}")

llm = ChatGoogleGenerativeAI(
    model=active_model,
    temperature=0.3,
    max_retries=2
)

search_tool = DuckDuckGoSearchRun()

# --- 2. DEFINE STATE ---
class AgentState(TypedDict):
    company_name: str
    target_role: str
    research_data: str
    strategy_points: str
    final_email: str
    messages: List[str]

# --- 3. AGENTS (With Rate Limit Protection) ---

def research_agent(state: AgentState):
    print(f"🔎 Researching: {state['company_name']}...")
    query = f"latest strategic business news {state['company_name']} 2024 2025"
    try:
        # Adding a timeout wrapper could be useful here, but basic invoke is usually fine
        search_result = search_tool.invoke(query)
    except Exception as e:
        search_result = f"Search unavailable: {e}"

    time.sleep(1) # Safety pause for Free Tier
    return {
        "research_data": search_result,
        "messages": [f"Research complete"]
    }

def analysis_agent(state: AgentState):
    print("🧠 Analyzing data...")
    prompt = f"""
    Analyze this news for {state['company_name']}:
    {state['research_data']}

    Identify 2 key business problems they are trying to solve.
    Return ONLY bullet points.
    """
    try:
        response = llm.invoke(prompt)
        content = response.content
    except Exception as e:
        content = f"Analysis skipped: {e}"

    time.sleep(1)
    return {
        "strategy_points": content,
        "messages": [f"Analysis complete"]
    }

def copywriter_agent(state: AgentState):
    print("✍️ Drafting email...")
    prompt = f"""
    Write a cold email to a {state['target_role']} at {state['company_name']}.
    Mention these priorities to build relevance:
    {state['strategy_points']}

    Keep it under 100 words.
    """
    try:
        response = llm.invoke(prompt)
        content = response.content
    except Exception as e:
        content = f"Drafting skipped: {e}"

    return {
        "final_email": content,
        "messages": [f"Drafting complete"]
    }

# --- 4. COMPILE ---
workflow = StateGraph(AgentState)
workflow.add_node("researcher", research_agent)
workflow.add_node("analyst", analysis_agent)
workflow.add_node("copywriter", copywriter_agent)

workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "copywriter")
workflow.add_edge("copywriter", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("✅ Agent System Ready.")

🤖 Initializing Agent with Model: gemini-2.5-flash
✅ Agent System Ready.


In [2]:
# @title ⚡ Run the Agent
target_company = "Spotify" # @param {type:"string"}
target_person_role = "VP of Marketing" # @param {type:"string"}

config = {"configurable": {"thread_id": "final_run_v2"}}

inputs = {
    "company_name": target_company,
    "target_role": target_person_role,
    "research_data": "",
    "strategy_points": "",
    "final_email": "",
    "messages": []
}

print(f"🚀 Starting sequence for {target_company}...")
result = app.invoke(inputs, config=config)

print("\n" + "="*50)
print("🎯 FINAL EMAIL DRAFT")
print("="*50)
print(result['final_email'])

🔄 Verifying model access...
✅ Selected Stable Model: gemini-2.0-flash-exp
✅ Agent System Compiled.


In [6]:
# @title ⚡ Run Your Agent (Test)
target_company = "Spotify" # @param {type:"string"}
target_person_role = "VP of Marketing" # @param {type:"string"}

config = {"configurable": {"thread_id": "session_final_v1"}}

inputs = {
    "company_name": target_company,
    "target_role": target_person_role,
    "research_data": "",
    "strategy_points": "",
    "final_email": "",
    "messages": []
}

print(f"🚀 Starting sequence for {target_company}...\n")
try:
    result = app.invoke(inputs, config=config)

    print("\n" + "="*50)
    print("🎯 FINAL OUTPUT: COLD OUTREACH DRAFT")
    print("="*50)
    print(result['final_email'])
    print("="*50)
except Exception as e:
    print(f"❌ Execution failed: {e}")

🚀 Starting sequence for Spotify...

🔎 Researching: Spotify...
🧠 Analyzing data...
✍️ Drafting email...

🎯 FINAL OUTPUT: COLD OUTREACH DRAFT
Subject: Accelerating Spotify's Sustainable Growth

Hi [VP's Name],

I imagine a key focus for you is driving sustainable revenue growth while building Spotify into an enduring, truly great business for the long term.

My firm specializes in helping marketing VPs achieve these exact objectives by uncovering new pathways to accelerate growth and secure lasting market leadership.

Would you be open to a brief 15-minute discussion next week to see if there's a fit?

Best,
[My Name]


**New Code**

In [44]:
# @title  Step 1: Install Dependencies & Configure Gemini

!pip install -U google-generativeai langchain-google-genai langgraph langchain_community duckduckgo-search requests > /dev/null

import os
import getpass
import time
import json
from typing import TypedDict, List, Dict, Any

import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate


In [45]:
# @title  Setup Gemini API Key & Pick Model

# 1. Get / set API key (DO NOT hardcode in code you commit)
if "GOOGLE_API_KEY" not in os.environ or not os.environ["GOOGLE_API_KEY"]:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google Gemini API Key: ")

api_key = os.environ["GOOGLE_API_KEY"]

def pick_gemini_model(api_key: str) -> str:
    """
    Pick a reasonable Gemini model for chat.
    If the API call fails, fall back to a safe default.
    """
    base_url = "https://generativelanguage.googleapis.com/v1beta/models"
    try:
        resp = requests.get(base_url, params={"key": api_key}, timeout=10)
        resp.raise_for_status()
        models = resp.json().get("models", [])
        # Prefer latest flash/pro chat models
        preferred_prefixes = [
            "gemini-1.5-flash",
            "gemini-1.5-pro",
            "gemini-1.0-pro",
            "gemini-1.0-pro-vision",
        ]
        for prefix in preferred_prefixes:
            for m in models:
                if m["name"].startswith(prefix):
                    print(f" Using Gemini model: {m['name']}")
                    return m["name"]
    except Exception as e:
        print(f" Model discovery failed, using default. Reason: {e}")

    # Safe default (update if needed)
    default_model = "gemini-1.5-flash"
    print(f" Falling back to default Gemini model: {default_model}")
    return default_model

# 🔑 Choose a valid Gemini chat model explicitly

# Common good choices (try these in order if you get 404):
# - "gemini-1.5-flash-002"
# - "gemini-1.5-flash-001"
# - "gemini-1.5-flash-8b"

ACTIVE_MODEL = "gemini-2.5-flash"
print(f" Using Gemini model: {ACTIVE_MODEL}")



 Using Gemini model: gemini-2.5-flash


In [46]:
# @title  Define Agent State, LLM, and Search Tool

class AgentState(TypedDict, total=False):
    company_name: str
    target_role: str
    research_data: str          # raw search output / summary
    strategy_points: str        # bullet points of business problems
    final_email: str            # final outreach email
    evaluation: str             # evaluation JSON or text
    metrics: Dict[str, float]   # simple timing metrics


# Shared Gemini LLM for all agents
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model=ACTIVE_MODEL,
    temperature=0.4,
    google_api_key=os.environ["GOOGLE_API_KEY"],  # optional if env var already set
)

# Web search tool
search_tool = DuckDuckGoSearchRun()


In [47]:
# @title  Research Agent

research_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a B2B research assistant. Given a company name and raw web search "
        "results, produce a concise summary of recent strategic business news or "
        "initiatives (last 1-2 years). Focus on product launches, market expansion, "
        "monetization shifts, and leadership statements. Avoid generic company descriptions."
    ),
    (
        "user",
        "Company: {company_name}\n\nRaw search results:\n{search_results}\n\n"
        "Summarize the most relevant business and strategic updates in under 250 words."
    ),
])
research_chain = research_prompt | llm

def research_agent(state: AgentState) -> AgentState:
    t0 = time.time()
    company = state["company_name"]
    print(f" Research Agent: Searching news for {company}...")

    # Use DuckDuckGo search as tool
    query = f"latest strategic business news {company} 2024 2025"
    raw_results = search_tool.invoke(query)

    # Make it human-readable for the LLM
    search_text = raw_results if isinstance(raw_results, str) else json.dumps(raw_results, ensure_ascii=False)[:4000]

    summary = research_chain.invoke({
        "company_name": company,
        "search_results": search_text,
    }).content

    duration = time.time() - t0
    metrics = dict(state.get("metrics", {}))
    metrics["research_duration_sec"] = duration

    print(f" Research Agent done in {duration:.2f}s\n")

    return {
        "research_data": summary,
        "metrics": metrics,
    }


In [48]:
# @title  Analysis Agent

analysis_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a strategy analyst. Given a summary of recent news about a company, "
        "identify exactly 2 key business problems or strategic priorities they seem to be focused on.\n\n"
        "Return ONLY bullet points, no extra commentary."
    ),
    (
        "user",
        "Company: {company_name}\n\nNews summary:\n{research_data}\n\n"
        "Identify 2 key business problems or strategic priorities."
    ),
])
analysis_chain = analysis_prompt | llm

def analysis_agent(state: AgentState) -> AgentState:
    t0 = time.time()
    print(" Analysis Agent: Extracting key business problems...")

    company = state["company_name"]
    research_data = state["research_data"]

    strategy_points = analysis_chain.invoke({
        "company_name": company,
        "research_data": research_data,
    }).content

    duration = time.time() - t0
    metrics = dict(state.get("metrics", {}))
    metrics["analysis_duration_sec"] = duration

    print(f" Analysis Agent done in {duration:.2f}s\n")

    return {
        "strategy_points": strategy_points,
        "metrics": metrics,
    }


In [49]:
# @title  Copywriter Agent

copywriter_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a world-class B2B SDR copywriter. Your goal is to write short, "
        "highly personalized cold emails that reference the prospect's company priorities explicitly.\n\n"
        "Constraints:\n"
        "- Under 100 words.\n"
        "- Conversational, not overly formal.\n"
        "- Reference the business problems explicitly.\n"
        "- Clear call to action for a quick call or reply."
    ),
    (
        "user",
        "Write a cold email to the {target_role} at {company_name}.\n\n"
        "Use these business problems / priorities for personalization:\n{strategy_points}\n\n"
        "Return only the email body, no subject line, no extra commentary."
    ),
])
copywriter_chain = copywriter_prompt | llm

def copywriter_agent(state: AgentState) -> AgentState:
    t0 = time.time()
    company = state["company_name"]
    role = state["target_role"]
    print(f" Copywriter Agent: Drafting email for {role} at {company}...")

    email_body = copywriter_chain.invoke({
        "company_name": company,
        "target_role": role,
        "strategy_points": state["strategy_points"],
    }).content

    duration = time.time() - t0
    metrics = dict(state.get("metrics", {}))
    metrics["copywriter_duration_sec"] = duration

    print(f" Copywriter Agent done in {duration:.2f}s\n")

    return {
        "final_email": email_body,
        "metrics": metrics,
    }


In [50]:
# @title  Evaluator Agent (Scoring & Feedback) – FIXED ESCAPING

from langchain_core.prompts import ChatPromptTemplate

evaluator_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        # NOTE: all literal { } in the JSON example are escaped as {{ }}
        "You are an expert sales coach and copy evaluator.\n"
        "Given the company, target role, business problems, and a cold email, "
        "evaluate the email and return STRICT JSON with this structure:\n\n"
        "{{\n"
        '  "score_relevance": <integer 0-10>,\n'
        '  "score_personalization": <integer 0-10>,\n'
        '  "score_clarity": <integer 0-10>,\n'
        '  "overall_score": <integer 0-10>,\n'
        '  "strengths": ["..."],\n'
        '  "weaknesses": ["..."],\n'
        '  "suggestions": ["..."]\n'
        "}}\n\n"
        "Return only valid JSON, no extra text."
    ),
    (
        "user",
        "Company: {company_name}\n"
        "Target role: {target_role}\n\n"
        "Business problems / priorities:\n{strategy_points}\n\n"
        "Cold email:\n{final_email}"
    ),
])

evaluator_chain = evaluator_prompt | llm

def evaluator_agent(state: AgentState) -> AgentState:
    t0 = time.time()
    print(" Evaluator Agent: Scoring email quality...")

    raw_eval = evaluator_chain.invoke({
        "company_name": state["company_name"],
        "target_role": state["target_role"],
        "strategy_points": state["strategy_points"],
        "final_email": state["final_email"],
    }).content

    duration = time.time() - t0
    metrics = dict(state.get("metrics", {}))
    metrics["evaluator_duration_sec"] = duration

    # Try to parse JSON; if it fails, keep raw text so notebook doesn't crash
    try:
        parsed = json.loads(raw_eval)
        pretty = json.dumps(parsed, indent=2)
    except Exception:
        pretty = raw_eval

    print(f" Evaluator Agent done in {duration:.2f}s\n")

    return {
        "evaluation": pretty,
        "metrics": metrics,
    }


In [51]:
# @title  Build Multi-Agent Graph with MemorySaver

# Build the StateGraph
graph = StateGraph(AgentState)

graph.add_node("researcher", research_agent)
graph.add_node("analyst", analysis_agent)
graph.add_node("copywriter", copywriter_agent)
graph.add_node("evaluator", evaluator_agent)

graph.set_entry_point("researcher")
graph.add_edge("researcher", "analyst")
graph.add_edge("analyst", "copywriter")
graph.add_edge("copywriter", "evaluator")
graph.add_edge("evaluator", END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)


print(" LangGraph app compiled with MemorySaver.")


 LangGraph app compiled with MemorySaver.


In [52]:
# @title  Run DealCloser Once (New Session)

target_company = "Spotify"        # @param {type:"string"}
target_person_role = "VP of Marketing"   # @param {type:"string"}

thread_id = "dealcloser_demo_v1"  # you can change per run / per account
config = {"configurable": {"thread_id": thread_id}}

inputs: AgentState = {
    "company_name": target_company,
    "target_role": target_person_role,
    "research_data": "",
    "strategy_points": "",
    "final_email": "",
    "evaluation": "",
    "metrics": {},
}

print(f" Starting sequence for {target_company} ({target_person_role})...\n")

t0 = time.time()
result: AgentState = app.invoke(inputs, config=config)
total_duration = time.time() - t0

print("\n" + "=" * 60)
print(" FINAL OUTPUT: COLD OUTREACH DRAFT")
print("=" * 60)
print(result["final_email"])
print("=" * 60 + "\n")

print(" EVALUATION (JSON-like):")
print(result["evaluation"])
print("\n" + "=" * 60)

print(" METRICS:")
metrics = dict(result.get("metrics", {}))
metrics["total_pipeline_duration_sec"] = total_duration
print(json.dumps(metrics, indent=2))


 Starting sequence for Spotify (VP of Marketing)...

 Research Agent: Searching news for Spotify...
 Research Agent done in 6.96s

 Analysis Agent: Extracting key business problems...
 Analysis Agent done in 1.78s

 Copywriter Agent: Drafting email for VP of Marketing at Spotify...
 Copywriter Agent done in 6.88s

 Evaluator Agent: Scoring email quality...
 Evaluator Agent done in 13.66s


 FINAL OUTPUT: COLD OUTREACH DRAFT
Hi [VP Marketing Name],

I imagine **accelerating subscriber growth and monetization** is a constant focus for you at Spotify. I also understand the critical need for **ensuring long-term business sustainability and operational efficiency**.

My team helps marketing leaders tackle these exact challenges, and I believe we could offer some valuable insights for Spotify.

Would you be open to a quick 15-minute chat next week to explore this further?

 EVALUATION (JSON-like):
```json
{
  "score_relevance": 9,
  "score_personalization": 6,
  "score_clarity": 7,
  "overal